In [ ]:
import pandas as pd
import csv

#import file csv
file = 'dataset_tweet_2.csv'

token_data = open(file)
tokens = csv.reader(token_data, delimiter=';')
tweets = []
label = []
for row in tokens:
    tweets.append(row[0])
    label.append(int(row[1].replace(',','')))

df = pd.DataFrame(columns=['tweets','label'])
df['tweets'] = tweets
df['label'] = label

print (df)

                                               tweets  label
0   rt @napqilla: no 1, 3 ambisinya menguasai raky...      1
1   rt @pandji: nah gue pikir sentimen petahana ok...      1
2   rt @pandji: urutan pertama best moment #debat2...      1
3   rt @pandji: ini artikel yg menjelaskan ternyat...      1
4   rt @mrtampi: agus makin santai.\nahok makin sa...      0
..                                                ...    ...
76  rt @pandji: nah gue pikir sentimen petahana ok...      0
77  rt @josua_tm: ibu sylvi adalah contoh bahwa wa...      1
78  besok saya ajak kesana saja, saya udah survei ...      1
79  benerr bgt.. dan tidak mengajak penonton ikut ...      1
80  rt @gandy_koz: pak anis,kl pas libur lebaran i...      1

[81 rows x 2 columns]


In [ ]:
!pip nltk
!pip install sastrawi
import re,string
#import nltk.stem as stemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

clean_tweets = []
for tweet in tweets:
    def hapus_tanda(tweet): 
        tanda_baca = set(string.punctuation)
        tweet = ''.join(ch for ch in tweet if ch not in tanda_baca)
        return tweet
    
    def hapus_katadouble(s): 
        pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
        return pattern.sub(r"\1\1", s)

    tweet=tweet.lower()
    tweet = re.sub(r'\\u\w\w\w\w', '', tweet)
    tweet=re.sub(r'http\S+','',tweet)
    #hapus @username
    tweet=re.sub('@[^\s]+','',tweet)
    #hapus #tagger 
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #hapus tanda baca
    tweet=hapus_tanda(tweet)
    #hapus angka dan angka yang berada dalam string 
    tweet=re.sub(r'\w*\d\w*', '',tweet).strip()
    #hapus repetisi karakter 
    tweet=hapus_katadouble(tweet)
    #stemming
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    tweet = stemmer.stem(tweet)
    clean_tweets.append(tweet)

df['clean'] = clean_tweets
print(df.head())

ERROR: unknown command "nltk"
     |████████████████████████████████| 215kB 8.1MB/s 
                                              tweets  ...                                              clean
0  rt @napqilla: no 1, 3 ambisinya menguasai raky...  ...  rt no ambisi kuasa rakyat ambisi layan rakyat ...
1  rt @pandji: nah gue pikir sentimen petahana ok...  ...  rt nah gue pikir sentimen tahana oke di malam ...
2  rt @pandji: urutan pertama best moment #debat2...  ...  rt urut pertama best moment pak basuki misahin...
3  rt @pandji: ini artikel yg menjelaskan ternyat...  ...  rt ini artikel yg jelas nyata di yg dapet resp...
4  rt @mrtampi: agus makin santai.\nahok makin sa...  ...  rt agus makin santainahok makin santunnanies m...

[5 rows x 3 columns]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB

vectorizer = TfidfVectorizer (max_features=2500)
model_g = GaussianNB()

v_data = vectorizer.fit_transform(df['clean']).toarray()

print (v_data)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(v_data, df['label'], test_size=0.2, random_state=0)
model_g.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_preds = model_g.predict(X_test)

print(confusion_matrix(y_test,y_preds))
print(classification_report(y_test,y_preds))
print('nilai akurasinya adalah ',accuracy_score(y_test, y_preds))

[[8 4]
 [1 4]]
              precision    recall  f1-score   support

           0       0.89      0.67      0.76        12
           1       0.50      0.80      0.62         5

    accuracy                           0.71        17
   macro avg       0.69      0.73      0.69        17
weighted avg       0.77      0.71      0.72        17

nilai akurasinya adalah  0.7058823529411765


In [ ]:
tweet = ''
v_data = vectorizer.transform([tweet]).toarray()
y_preds = model_g.predict(v_data)

if y_preds == 1:
    print('Positif')
else:
    print('Negatif')

Positif
